In [1]:
from ctransformers import AutoModelForCausalLM
from huggingface_hub.hf_api import HfApi
from huggingface_hub.utils import logging
import torch
import os  
import pandas as pd  
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling  
from datasets import Dataset  

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Set environment variable for cache directory to a local permitted directory 

os.environ['TRANSFORMERS_CACHE'] = './transformers_cache'  
os.environ['HF_HOME'] = './hf_home'

In [ ]:
# Load CSV data  

print("Loading CSV file...")  
df = pd.read_csv('Combined Data.csv') 

In [ ]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

c:\Users\mward\anaconda3\envs\dev\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mward\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Check for missing statements and filter them out  
df = df.dropna(subset=['statement'])  
print("Data after removing missing values: " + str(df.shape))  

NameError: name 'df' is not defined

In [ ]:
# Create a Hugging Face Dataset from the DataFrame (using just the 'statement' column)  
dataset = Dataset.from_pandas(df[['statement']])  

In [ ]:
# Load tokenizer and model (using your working code)  
print("Loading tokenizer and model...")  
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")  
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")  
print("Tokenizer and model loaded successfully.")  

In [ ]:
# Define tokenization function using the "statement" column  
def tokenize_function(examples):  
    return tokenizer(examples["statement"], padding="max_length", truncation=True, max_length=128) 

In [ ]:
# Tokenize the dataset  
print("Tokenizing dataset...")  
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)  
print("Dataset tokenized successfully.")  

In [ ]:
# Setup data collator (for causal LM, mlm is False)  
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  

In [ ]:
# Set up training arguments (adjust hyperparameters as needed)  
training_args = TrainingArguments(  
    output_dir="./happy_brain",  
    overwrite_output_dir=True,  
    num_train_epochs=1,  # Adjust the number of epochs as needed  
    per_device_train_batch_size=4,  
    save_steps=1000,  
    save_total_limit=2,  
    logging_steps=100,  
    evaluation_strategy="no"  
) 

In [ ]:
# Initialize the Trainer  
trainer = Trainer(  
    model=model,  
    args=training_args,  
    train_dataset=tokenized_dataset,  
    data_collator=data_collator,  
)

print("Trainer initialized. Ready to start training.")  
print("Note: Full training requires significant time and resources.") 

# Uncomment the following line to start training:  
# trainer.train()  
  
# Save the model and tokenizer after training (uncomment when trainer.train() is executed)  
# model.save_pretrained("./happy_brain")  
# tokenizer.save_pretrained("./happy_brain")  

In [ ]:
# For demonstration, here's a sample inference function using the base model:  
def generate_response(input_text, max_length=50):  
    inputs = tokenizer(input_text, return_tensors="pt")  
    outputs = model.generate(  
        inputs.input_ids,   
        max_length=max_length,  
        num_return_sequences=1,  
        temperature=0.7  
    )  
    return tokenizer.decode(outputs[0], skip_special_tokens=True)  

print("\nExample inference (using base model):")  
sample_input = "I've been feeling really anxious lately and can't sleep."  
print("Input: " + sample_input)  
print("Response: " + generate_response(sample_input))  